# GBT

In [3]:
import os  
import sys  
spark_name = os.environ.get('SPARK_HOME',None)  
if not spark_name:  
    raise ValueErrorError('spark环境没有配置好')  
sys.path.insert(0,os.path.join(spark_name,'python'))  
sys.path.insert(0,os.path.join(spark_name,'python/lib/py4j-0.10.7-src.zip'))  
exec(open(os.path.join(spark_name,'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.1
      /_/

Using Python version 3.6.4 (default, Jan 16 2018 10:22:32)
SparkSession available as 'spark'.


In [27]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pprint import pprint

In [28]:
# data = spark.read.text('sample_libsvm_data.txt')
# data

In [29]:
# data.first()

In [33]:
# Load and parse the data file, converting it to a DataFrame
# data = spark.read.load('./sample_libsvm_data.txt', format='text')  # load的话必须要带上format格式，并且要和文件格式匹配

# 读成libsvm格式？？
data = spark.read.load('./sample_libsvm_data.txt', format='libsvm')  # 此处libsvm格式读写出错


Py4JJavaError: An error occurred while calling o596.load.
: java.lang.UnsupportedOperationException: empty collection
	at org.apache.spark.rdd.RDD$$anonfun$reduce$1$$anonfun$apply$36.apply(RDD.scala:1031)
	at org.apache.spark.rdd.RDD$$anonfun$reduce$1$$anonfun$apply$36.apply(RDD.scala:1031)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.rdd.RDD$$anonfun$reduce$1.apply(RDD.scala:1031)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:363)
	at org.apache.spark.rdd.RDD.reduce(RDD.scala:1011)
	at org.apache.spark.mllib.util.MLUtils$.computeNumFeatures(MLUtils.scala:94)
	at org.apache.spark.ml.source.libsvm.LibSVMFileFormat$$anonfun$1.apply$mcI$sp(LibSVMRelation.scala:104)
	at org.apache.spark.ml.source.libsvm.LibSVMFileFormat$$anonfun$1.apply(LibSVMRelation.scala:95)
	at org.apache.spark.ml.source

In [31]:
data

DataFrame[value: string]

In [32]:
data.first()

Row(value='0 128:51 129:159 130:253 131:159 132:50 155:48 156:238 157:252 158:252 159:252 160:237 182:54 183:227 184:253 185:252 186:239 187:233 188:252 189:57 190:6 208:10 209:60 210:224 211:252 212:253 213:252 214:202 215:84 216:252 217:253 218:122 236:163 237:252 238:252 239:252 240:253 241:252 242:252 243:96 244:189 245:253 246:167 263:51 264:238 265:253 266:253 267:190 268:114 269:253 270:228 271:47 272:79 273:255 274:168 290:48 291:238 292:252 293:252 294:179 295:12 296:75 297:121 298:21 301:253 302:243 303:50 317:38 318:165 319:253 320:233 321:208 322:84 329:253 330:252 331:165 344:7 345:178 346:252 347:240 348:71 349:19 350:28 357:253 358:252 359:195 372:57 373:252 374:252 375:63 385:253 386:252 387:195 400:198 401:253 402:190 413:255 414:253 415:196 427:76 428:246 429:252 430:112 441:253 442:252 443:148 455:85 456:252 457:230 458:25 467:7 468:135 469:253 470:186 471:12 483:85 484:252 485:223 494:7 495:131 496:252 497:225 498:71 511:85 512:252 513:145 521:48 522:165 523:252 524

In [26]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol='label', outputCol='indexedLabel').fit(data)

# Automatically identify categorical features, and index them.
# Set maxCategories so features with > 4 distinct values are treated as continuous.


IllegalArgumentException: 'Field "label" does not exist.\nAvailable fields: value'